# 01 – Local CLI in Docker

This notebook demonstrates the core features of Dockyter using a local Docker
installation:

- `%%docker` – run a whole cell in a single container.
- `%docker` – configure Docker mode for `!` commands.
- `%docker_status` – inspect the current configuration.
- `%docker_off` / `%docker_on` – enable/disable `!` redirection.

**Prerequisites**

- Docker CLI available (for example, `docker --version` works in a terminal).
- The `dockyter` extension installed in this environment (`pip install dockyter`).


In [11]:
# Optional: check that Docker is available
!docker --version

Forbidden Docker flag detected: --network=host


In [12]:
# Load the Dockyter IPython extension
%load_ext dockyter

The dockyter extension is already loaded. To reload it, use:
  %reload_ext dockyter


## 1. Basic `%%docker` usage (single container per cell)

`%%docker` runs the **entire cell** in a single container. All commands share
the same shell state (e.g. `cd` affects the following `ls`).

Here we use the `ubuntu:22.04` image as a simple example.

In [13]:
%%docker ubuntu:22.04
echo "Hello from inside ubuntu:22.04"
pwd
ls /

Hello from inside ubuntu:22.04
/
bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var



## 2. `%docker` + `!` redirection

`%docker` configures Dockyter so that each subsequent `!cmd` is executed inside
a container with the given image and arguments.

It also automatically turns on Docker mode for `!`.

In [14]:
%docker ubuntu:22.04

%docker_status

Connected

=== Dockyter Backend Status ===
Dockyter backend type: Docker
Dockyter backend status: available
Dockyter backend status: Docker is installed and the daemon is running.
Dockyer redirection for '!': on
Current docker args: ubuntu:22.04


In [15]:
!echo "This was executed inside ubuntu:22.04 via Dockyter"

This was executed inside ubuntu:22.04 via Dockyter



## 3. `!` commands do NOT share shell state

Each `!cmd` is run in a **fresh container**, so shell state does not persist
between `!` calls.

This is different from a `%%docker` cell, where all lines share the same
container and shell.

Below, `cd /tmp` does not affect the next `!pwd`, because they are separate
container runs.


In [16]:
!cd /tmp
!pwd

/



If you need multiple commands to share state (directories, environment
variables, shell functions, etc.), use a single `%%docker` cell instead.

## 4. `%docker_off` and `%docker_on`

- `%docker_off` restores the original `!` behaviour (no Docker redirection).
- `%docker_on` re-activates Docker mode for `!` using the last configured
  image/arguments.


In [17]:
%docker_off

!dir
%docker_status

bin   dev  home  lib32	libx32	mnt  proc  run	 srv  tmp  var
boot  etc  lib	 lib64	media	opt  root  sbin  sys  usr

=== Dockyter Backend Status ===
Dockyter backend type: Docker
Dockyter backend status: available
Dockyter backend status: Docker is installed and the daemon is running.
Dockyer redirection for '!': off
Current docker args: ubuntu:22.04


In [18]:
%docker_on

!dir
%docker_status

bin   dev  home  lib32	libx32	mnt  proc  run	 srv  tmp  var
boot  etc  lib	 lib64	media	opt  root  sbin  sys  usr

=== Dockyter Backend Status ===
Dockyter backend type: Docker
Dockyter backend status: available
Dockyter backend status: Docker is installed and the daemon is running.
Dockyer redirection for '!': on
Current docker args: ubuntu:22.04
